# TODOS

~~1. Fix nonsense output - caused by missing backquote" at end of sentence (16-5-23)~~

~~2. Run with model_type = "" (not CAUSAL_LM)~~

3. Do more prompts - generic knowledge like Hemingway

~~3. Train with Linkedin data and check result~~

# Necessary to run these cells before running Specific folders below




In [3]:
!conda list

# packages in environment at /Users/ariellubonja/miniconda3:
#
# Name                    Version                   Build  Channel
asciitree                 0.3.3                    pypi_0    pypi
blas                      1.0                    openblas  
boltons                   23.0.0           py39hca03da5_0  
brotlipy                  0.7.0           py39h1a28f6b_1002  
ca-certificates           2023.01.10           hca03da5_0  
certifi                   2023.5.7         py39hca03da5_0  
cffi                      1.15.0           py39h22df2f2_1  
charset-normalizer        2.0.4              pyhd3eb1b0_0  
click                     8.0.4            py39hca03da5_0  
conda                     23.3.1           py39hca03da5_0  
conda-content-trust       0.1.1              pyhd3eb1b0_0  
conda-package-handling    1.8.1            py39h1a28f6b_0  
cryptography              37.0.1           py39h834c97f_0  
dataclasses               0.8                pyh6d0b6a4_7  
entrypoints           

In [3]:
!pip install -q bitsandbytes datasets accelerate loralib
!pip install -q git+https://github.com/huggingface/transformers.git@main git+https://github.com/huggingface/peft.git


[notice] A new release of pip is available: 23.0 -> 23.1.2
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip is available: 23.0 -> 23.1.2
[notice] To update, run: pip install --upgrade pip


In [2]:
from huggingface_hub import notebook_login

notebook_login()

ModuleNotFoundError: No module named 'huggingface_hub'

In [3]:
!nvidia-smi -L

GPU 0: Tesla T4 (UUID: GPU-d0099633-cc39-6b3f-2245-38974f60989d)


In [4]:
def print_trainable_parameters(model):
    """
    Prints the number of trainable parameters in the model.
    """
    trainable_params = 0
    all_param = 0
    for _, param in model.named_parameters():
        all_param += param.numel()
        if param.requires_grad:
            trainable_params += param.numel()
    print(
        f"trainable params: {trainable_params} || all params: {all_param} || trainable%: {100 * trainable_params / all_param}"
    )


def merge_columns(example):
    """
    Creates a Dictionary entry called "Prediction"
    It consists of Quote + ->: + Tags
    """
    example["prediction"] = example["quote"] + " ->: " + str(example["tags"])
    return example
  


def test_model_output(model):
    batch = tokenizer("“Training models with PEFT and LoRa is cool” ->: ", return_tensors='pt').to('cuda')

    with torch.cuda.amp.autocast():
        output_tokens = model.generate(**batch, max_new_tokens=50)


    # print("\n\nRaw output: ", output_tokens)
    print('\n\n', tokenizer.decode(output_tokens[0], skip_special_tokens=True))

    # 2nd
    batch = tokenizer("“The world is your oytster” ->: ", return_tensors='pt').to('cuda')
    with torch.cuda.amp.autocast():
        output_tokens = model.generate(**batch, max_new_tokens=50)


    print('\n\n', tokenizer.decode(output_tokens[0], skip_special_tokens=True))


    # 3rd test
    batch = tokenizer("“Too many books, too little time”->: ", return_tensors='pt').to('cuda')
    with torch.cuda.amp.autocast():
        output_tokens = model.generate(**batch, max_new_tokens=50)

    # print("\n\nRaw output: ", output_tokens)
    print('\n\n', tokenizer.decode(output_tokens[0], skip_special_tokens=True))

In [5]:
# Can't run this after %cd Google Drive for some reason - gets torch import error
import os
os.environ["CUDA_VISIBLE_DEVICES"]="0"
import torch
import torch.nn as nn
import bitsandbytes as bnb
from transformers import AutoTokenizer, AutoConfig, AutoModelForCausalLM
import transformers


===================================BUG REPORT===================================
Welcome to bitsandbytes. For bug reports, please run

python -m bitsandbytes

 and submit this information together with your error trace to: https://github.com/TimDettmers/bitsandbytes/issues
bin /usr/local/lib/python3.10/dist-packages/bitsandbytes/libbitsandbytes_cuda118.so
CUDA_SETUP: WARNING! libcudart.so not found in any environmental path. Searching in backup paths...
CUDA SETUP: CUDA runtime path found: /usr/local/cuda/lib64/libcudart.so
CUDA SETUP: Highest compute capability among GPUs detected: 7.5
CUDA SETUP: Detected CUDA version 118
CUDA SETUP: Loading binary /usr/local/lib/python3.10/dist-packages/bitsandbytes/libbitsandbytes_cuda118.so...


/usr/local/lib/python3.10/dist-packages/bitsandbytes/cuda_setup/main.py:145: UserWarning: /usr/lib64-nvidia did not contain ['libcudart.so', 'libcudart.so.11.0', 'libcudart.so.12.0'] as expected! Searching further paths...
  warn(msg)
/usr/local/lib/python3.10/dist-packages/bitsandbytes/cuda_setup/main.py:145: UserWarning: WARNING: The following directories listed in your path were found to be non-existent: {PosixPath('/sys/fs/cgroup/memory.events /var/colab/cgroup/jupyter-children/memory.events')}
  warn(msg)
/usr/local/lib/python3.10/dist-packages/bitsandbytes/cuda_setup/main.py:145: UserWarning: WARNING: The following directories listed in your path were found to be non-existent: {PosixPath('//172.28.0.1'), PosixPath('http'), PosixPath('8013')}
  warn(msg)
/usr/local/lib/python3.10/dist-packages/bitsandbytes/cuda_setup/main.py:145: UserWarning: WARNING: The following directories listed in your path were found to be non-existent: {PosixPath('--logtostderr --listen_host=172.28.0.12 --

# Fine-Tune model w/ Linkedin Data `task_type="CAUSAL_LM"`

### Try model performance before fine-tuning

### <font color="red">Uncomment only 1 of the following lines! Select the size of the model</font>

In [6]:
model_size = "bigscience/bloom-560m"
# model_size = "bigscience/bloom-1b1" # 1.1 BN parames
# model_size = "bigscience/bloom-1b7" # 1.7 BN params
# model_size = "bigscience/bloom-3b"
# model_size = "bigscience/bloom-7b1"

In [7]:
model = AutoModelForCausalLM.from_pretrained(
    model_size,  #1.7B parames
    load_in_8bit=False, 
    device_map='auto',
)

tokenizer = AutoTokenizer.from_pretrained(model_size)

### Query the model before fine tuning

In [8]:
# From ChatGPT :)
general_prompts = [
    "Can you tell me a joke?",
    "What is the capital of France?",
    "Can you explain the concept of artificial intelligence?",
    "Tell me about the latest advancements in technology.",
    "Who is the current President of the United States?",
    "What are the potential applications of blockchain technology?",
    "Can you provide a summary of the book '1984' by George Orwell?",
    "What is the process of photosynthesis?",
    "Can you recommend a good restaurant in my area?",
    "Tell me about the life and contributions of Albert Einstein."
]

ds_prompts = [
    "Summarize skills necessary for a data scientist",
    "What are the key skills required for a data scientist?",
    "Can you provide an overview of a data scientist's role and responsibilities?",
    "What programming languages are commonly used by data scientists?",
    "What is the typical educational background of a data scientist?",
    "Can you explain the difference between data science and machine learning?",
    "What are some common tools and frameworks used by data scientists?",
    "What is the importance of statistical analysis in data science?",
    "Can you give examples of real-world applications of data science in various industries?",
    "What are the steps involved in the data science lifecycle?",
    "What are some challenges faced by data scientists in their work?"
]


In [9]:
for query in general_prompts:
  batch = tokenizer(query, return_tensors='pt').to('cuda')

  with torch.cuda.amp.autocast():
      output_tokens = model.generate(**batch, max_new_tokens=25)


  # print("\n\nRaw output: ", output_tokens)
  a = tokenizer.decode(output_tokens[0], skip_special_tokens=True)
  print('\n\n', a)


print("\n--------------------------------------------------------------------------------------------------------------")


for query in ds_prompts:
  batch = tokenizer(query, return_tensors='pt').to('cuda')

  with torch.cuda.amp.autocast():
      output_tokens = model.generate(**batch, max_new_tokens=25)


  # print("\n\nRaw output: ", output_tokens)
  a = tokenizer.decode(output_tokens[0], skip_special_tokens=True)
  print('\n\n', a)



 Can you tell me a joke?"

"I'm not joking," said the girl, with a smile. "I don't know what to
say.


 What is the capital of France?"

"It is the capital of the French Republic," said the Frenchman, with a
smile.

"


 Can you explain the concept of artificial intelligence?"
"It is a very complex concept. It is a very complex concept. It is a very complex concept. It


 Tell me about the latest advancements in technology.”
“We’re not going to be able to do that,” said the president. “We’re going to have


 Who is the current President of the United States?”
“Mr. President, I am the President of the United States.”
“Mr. President, I am


 What are the potential applications of blockchain technology? Blockchain technology is a new technology that is being developed by the blockchain community. Blockchain technology is a new technology that is being developed


 Can you provide a summary of the book '1984' by George Orwell? I have read it and I think it is a good book. I have read it 

 <font color="red">Load in 8bit = False. TODO fix</font>
 
 <font color="orange">560m model (not fine-tuned)</font>
 
 Q: Summarize skills necessary for a data scientist to work in a data-driven environment.
 
 A: The course is designed to help you develop a data-driven approach to data analysis and visualization. The course is designed to help you develop a data-driven approach to data analysis and visualization. The course is designed to help


 <font color="orange">1b1 model (not fine-tuned)</font>

 A: The course will cover the fundamentals of data science, including data modeling, data analysis, data visualization, and data science tools. The course will also cover the latest trends in data science, including machine learning


  <font color="orange">1b7 (1.7B) model (not fine-tuned)</font>

<font color="red">A: Empty for some reason??</font>

<font color="orange">3b model (not fine-tuned)</font>

  A: This is a great way to get a feel for the data science field and what it takes to be a data scientist.


 <font color="orange">7B model (not fine-tuned)</font>
 
 Q: Summarize skills necessary for a data scientist to work in a data-driven environment.
 
 <font color="red">A: Out of memory error on A100</font>

## Fine-Tuning

Skip if loading fine-tuned model from HF

In [11]:
# Upload JSON
import json

linkedin_data = ""
with open("./DataScientist_US_Entrylevel_Fulltime.json", 'r') as file:
    linkedin_data = json.load(file)



from datasets import Dataset
jobs_list = [{"job_desc":linkedin_data[s]['Job Description']} for s in list(linkedin_data.keys())]
dataset = Dataset.from_list(jobs_list)

dataset = dataset.train_test_split(test_size=0.2)

data_map = dataset.map(lambda samples: tokenizer(samples['job_desc']), batched=True)

Map:   0%|          | 0/114 [00:00<?, ? examples/s]

Map:   0%|          | 0/29 [00:00<?, ? examples/s]

<font color="red">To use your own data for model fine-tuning, you must first format your training and evaluation data into Spark DataFrames.</font>

https://docs.databricks.com/machine-learning/train-model/huggingface/load-data.html

In [12]:
# Freeze weights
for param in model.parameters():
  param.requires_grad = False  # freeze the model - train adapters later
  if param.ndim == 1:
    # cast the small parameters (e.g. layernorm) to fp32 for stability
    param.data = param.data.to(torch.float32)

model.gradient_checkpointing_enable()  # reduce number of stored activations
model.enable_input_require_grads()

class CastOutputToFloat(nn.Sequential):
  def forward(self, x): return super().forward(x).to(torch.float32) # This used to be 32
model.lm_head = CastOutputToFloat(model.lm_head)

In [13]:
from peft import LoraConfig, get_peft_model 

config = LoraConfig(
    r=16, #attention heads
    lora_alpha=32, #alpha scaling
    # target_modules=["q_proj", "v_proj"], #if you know the 
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM"
)

model = get_peft_model(model, config)
print_trainable_parameters(model)

trainable params: 1572864 || all params: 560787456 || trainable%: 0.2804741766549072


In [43]:
import transformers
trainer = transformers.Trainer(
    model=model, 
    train_dataset=data_map['train'],
    args=transformers.TrainingArguments(
        per_device_train_batch_size=1, 
        gradient_accumulation_steps=4,
        warmup_steps=100, 
        max_steps=200, 
        learning_rate=2e-4, 
        fp16=True,
        logging_steps=1,
        output_dir='outputs',
        push_to_hub=True
    ),
    data_collator=transformers.DataCollatorForLanguageModeling(tokenizer, mlm=False)
)
model.config.use_cache = False  # silence the warnings. Please re-enable for inference!

with torch.cuda.amp.autocast(): 
  trainer.train()

Cloning https://huggingface.co/Ariel4/outputs into local empty directory.


Download file pytorch_model.bin:   0%|          | 8.00k/2.09G [00:00<?, ?B/s]

Download file training_args.bin: 100%|##########| 3.81k/3.81k [00:00<?, ?B/s]

Clean file training_args.bin:  26%|##6       | 1.00k/3.81k [00:00<?, ?B/s]

Clean file pytorch_model.bin:   0%|          | 1.00k/2.09G [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:407: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Step,Training Loss
1,2.538000
2,2.540400
3,2.098000
4,2.294200
5,2.721400
6,2.105300
7,2.312400
8,1.885900
9,1.653400
10,2.185000


## Query the model

In [ ]:
batch = tokenizer("Summarize skills necessary for a data scientist", return_tensors='pt').to('cuda')

with torch.cuda.amp.autocast():
    output_tokens = model.generate(**batch, max_new_tokens=200)


# print("\n\nRaw output: ", output_tokens)
a = tokenizer.decode(output_tokens[0], skip_special_tokens=True)
print('\n\n', a)

 <font color="orange">560m model (fine-tuned)</font>
 
 Q: Summarize skills necessary for a data scientist to work in a data-driven environment.
 
 A: Identify and develop data models and algorithms to support data analysis. Identify and develop data visualization tools to support data analysis. Identify and develop data security and privacy policies to support data analysis. Develop and maintain data pipelines to support data analysis. Develop and maintain data analytics tools to support data analysis. Develop and maintain data pipelines to support data analysis. Develop and maintain data analytics tools to support data analysis. Develop and maintain data pipelines to support data analysis. Develop and maintain data pipelines to support data analysis. ...


  <font color="orange">1b1 model (fine-tuned)</font>

A: Describe the use of statistical methods and software to perform statistical analyses. Describe the use of data visualization to present data in a clear and understandable manner. Describe the use of data mining to identify patterns in large amounts of data. Describe the use of machine learning to predict future events. Describe the use of data visualization to present data in a clear and understandable manner. ...

<font color="orange">1b1 model (fine-tuned)</font>


<font color="orange">3b model (fine-tuned)</font>

A: Describe how you would use your skills to analyze data.Describe how you would use your skills to develop solutions.Describe how you would use your skills to communicate results.Describe how you would use your skills to collaborate with others.Describe how you would use your skills to apply your knowledge to real-world problems.Describe how you would use your skills to apply your knowledge to real-world problems.Describe how you would use your skills to apply your knowledge to real-world p

 <font color="orange">7B model (fine-tuned)</font>
 
 Q: Summarize skills necessary for a data scientist to work in a data-driven environment.
 
 A: 

## Optional - upload on HF Hub + Load from Hub

In [30]:
%cd /content/outputs

'/content/outputs'

In [42]:
!pwd

/content


In [41]:
# Don't replace "trainer" with "model". It does an incomplete push
trainer.push_to_hub("bloom-560m-lora-linkedin")

╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ in <cell line: 2>:2                                                                              │
│                                                                                                  │
│ /usr/local/lib/python3.10/dist-packages/transformers/trainer.py:3678 in push_to_hub              │
│                                                                                                  │
│   3675 │   │   # If a user calls manually `push_to_hub` with `self.args.push_to_hub = False`, w  │
│   3676 │   │   # it might fail.                                                                  │
│   3677 │   │   if not hasattr(self, "repo"):                                                     │
│ ❱ 3678 │   │   │   self.init_git_repo()                                                          │
│   3679 │   │                                                                                     │
│   3680 │   │   model_name = kwargs.pop("model_name", None)                                       │
│   3681 │   │   if model_name is None and self.args.should_save:                                  │
│                                                                                                  │
│ /usr/local/lib/python3.10/dist-packages/transformers/trainer.py:3534 in init_git_repo            │
│                                                                                                  │
│   3531 │   │   # Make sure the repo exists.                                                      │
│   3532 │   │   create_repo(repo_name, token=self.args.hub_token, private=self.args.hub_private_  │
│   3533 │   │   try:                                                                              │
│ ❱ 3534 │   │   │   self.repo = Repository(self.args.output_dir, clone_from=repo_name, token=sel  │
│   3535 │   │   except EnvironmentError:                                                          │
│   3536 │   │   │   if self.args.overwrite_output_dir and at_init:                                │
│   3537 │   │   │   │   # Try again after wiping output_dir                                       │
│                                                                                                  │
│ /usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_validators.py:120 in _inner_fn    │
│                                                                                                  │
│   117 │   │   if check_use_auth_token:                                                           │
│   118 │   │   │   kwargs = smoothly_deprecate_use_auth_token(fn_name=fn.__name__, has_token=ha   │
│   119 │   │                                                                                      │
│ ❱ 120 │   │   return fn(*args, **kwargs)                                                         │
│   121 │                                                                                          │
│   122 │   return _inner_fn  # type: ignore                                                       │
│   123                                                                                            │
│                                                                                                  │
│ /usr/local/lib/python3.10/dist-packages/huggingface_hub/repository.py:516 in __init__            │
│                                                                                                  │
│    513 │   │   │   self.huggingface_token = HfFolder.get_token()                                 │
│    514 │   │                                                                                     │
│    515 │   │   if clone_from is not None:                                                        │
│ ❱  516 │   │   │   self.clone_from(repo_url=clone_from)                                          │
│    517 │   │   else:                                       

# Load fine-tuned model from 🤗. Skip retraining

In [20]:
from transformers import AutoModel

# Load the model from the Hugging Face model hub
model_name = "Ariel4/bloom-560m-lora-linkedin"
model = AutoModel.from_pretrained(model_name)


╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ in <cell line: 5>:5                                                                              │
│                                                                                                  │
│ /usr/local/lib/python3.10/dist-packages/transformers/models/auto/auto_factory.py:472 in          │
│ from_pretrained                                                                                  │
│                                                                                                  │
│   469 │   │   │   )                                                                              │
│   470 │   │   elif type(config) in cls._model_mapping.keys():                                    │
│   471 │   │   │   model_class = _get_model_class(config, cls._model_mapping)                     │
│ ❱ 472 │   │   │   return model_class.from_pretrained(                                            │
│   473 │   │   │   │   pretrained_model_name_or_path, *model_args, config=config, **hub_kwargs,   │
│   474 │   │   │   )                                                                              │
│   475 │   │   raise ValueError(                                                                  │
│                                                                                                  │
│ /usr/local/lib/python3.10/dist-packages/transformers/modeling_utils.py:2500 in from_pretrained   │
│                                                                                                  │
│   2497 │   │   │   │   │   │   │   │   f" {variant}. Use `variant=None` to load this model from  │
│   2498 │   │   │   │   │   │   │   )                                                             │
│   2499 │   │   │   │   │   │   else:                                                             │
│ ❱ 2500 │   │   │   │   │   │   │   raise EnvironmentError(                                       │
│   2501 │   │   │   │   │   │   │   │   f"{pretrained_model_name_or_path} does not appear to hav  │
│   2502 │   │   │   │   │   │   │   │   f" {_add_variant(WEIGHTS_NAME, variant)}, {TF2_WEIGHTS_N  │
│   2503 │   │   │   │   │   │   │   │   f" {FLAX_WEIGHTS_NAME}."                                  │
╰──────────────────────────────────────────────────────────────────────────────────────────────────╯
OSError: Ariel4/bloom-560m-lora-linkedin does not appear to have a file named pytorch_model.bin, tf_model.h5, 
model.ckpt or flax_model.msgpack.

In [34]:
model = AutoModelForCausalLM.from_pretrained(
    "Ariel4/outputs",
    load_in_8bit=False, 
    device_map='auto',
)

tokenizer = AutoTokenizer.from_pretrained(model_size)

╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ /usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_errors.py:259 in                  │
│ hf_raise_for_status                                                                              │
│                                                                                                  │
│   256 │   </Tip>                                                                                 │
│   257 │   """                                                                                    │
│   258 │   try:                                                                                   │
│ ❱ 259 │   │   response.raise_for_status()                                                        │
│   260 │   except HTTPError as e:                                                                 │
│   261 │   │   error_code = response.headers.get("X-Error-Code")                                  │
│   262                                                                                            │
│                                                                                                  │
│ /usr/local/lib/python3.10/dist-packages/requests/models.py:960 in raise_for_status               │
│                                                                                                  │
│   957 │   │   │   http_error_msg = u'%s Server Error: %s for url: %s' % (self.status_code, rea   │
│   958 │   │                                                                                      │
│   959 │   │   if http_error_msg:                                                                 │
│ ❱ 960 │   │   │   raise HTTPError(http_error_msg, response=self)                                 │
│   961 │                                                                                          │
│   962 │   def close(self):                                                                       │
│   963 │   │   """Releases the connection back to the pool. Once this method has been             │
╰──────────────────────────────────────────────────────────────────────────────────────────────────╯
HTTPError: 404 Client Error: Not Found for url: https://huggingface.co/Ariel4/outputs/resolve/main/config.json

The above exception was the direct cause of the following exception:

╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ /usr/local/lib/python3.10/dist-packages/transformers/utils/hub.py:417 in cached_file             │
│                                                                                                  │
│    414 │   user_agent = http_user_agent(user_agent)                                              │
│    415 │   try:                                                                                  │
│    416 │   │   # Load from URL or cache if already cached                                        │
│ ❱  417 │   │   resolved_file = hf_hub_download(                                                  │
│    418 │   │   │   path_or_repo_id,                                                              │
│    419 │   │   │   filename,                                                                     │
│    420 │   │   │   subfolder=None if len(subfolder) == 0 else subfolder,                         │
│                                                                                                  │
│ /usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_validators.py:120 in _inner_fn    │
│                                                                                                  │
│   117 │   │   if check_use_auth_token:                                                           │
│   118 │   │   │   kwargs = smoothly_deprecate_use_auth_token(fn_name=fn.__name__, has_token=ha   │
│   119 │   │                                                                    